In [2]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:4])

gsk_


In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from pprint import pprint

template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    model="openai/gpt-oss-120b",
    temperature=0.7
)


In [6]:
from langchain_core.prompts import (
    FewShotChatMessagePromptTemplate,
    ChatPromptTemplate
)

from langchain.chains import LLMChain

# 1. 예시 프롬프트
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{news}"),
    ("ai", "키워드: {keywords}")
])

# 2. 학습 예시 데이터
examples = [
    {
        "news": "삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다.",
        "keywords": "삼성전자, 인공지능, 엔비디아"
    },
    {
        "news": "세계보건기구(WHO)는 최근 새로운 건강 위기에 대응하기 위해 국제 협력의 중요성을 강조했다.",
        "keywords": "세계보건기구, 건강위기, 국제협력"
    },
    {
        "news": "테슬라는 차세대 전기차 배터리를 공개하며, 주행 거리와 충전 속도를 획기적으로 개선했다고 밝혔다.",
        "keywords": "테슬라, 전기차, 배터리"
    }
]

# 3. FewShot 프롬프트
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

# 4. 최종 프롬프트
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 뉴스 키워드 추출 전문가입니다. 뉴스 기사에서 핵심 키워드 3개를 뽑으세요. 항상 '키워드: A, B, C' 형식을 따르세요."),
    few_shot_prompt,
    ("human", "{input}")
])

# 5. 체인 구성 (llm은 이미 작성 완료된 객체 활용)
chain = LLMChain(
    llm=llm,
    prompt=final_prompt,
    output_parser=StrOutputParser(),  # 문자열로 결과 받기
    verbose=True
)

# 6. 테스트 실행
test_news = "KT 무단 소액결제 피해가 당초 알려진 대로 서울 서남권·경기 일부 지역을 넘어 서울 서초구·동작구, 경기 고양시 일산동구 등에서도 일어난 것으로 뒤늦게 파악됐다. 20일 KT가 국회 과학기술정보방송통신위원회 소속 황정아 의원(더불어민주당)에게 제출한 인증 시간 기준 피해 지역 자료에 따르면 알려진 곳들 외에도 서울 동작구, 서초구, 고양시 일산동구가 포함돼 있다. KT가 처음 피해가 발생한 시점으로 지목한 지난달 5일 서울 동작구 일대에서 2건에 걸쳐 87만 9500원의 무단 결제가 발생했다. 이어 8일과 11일에는 서울 서초구에서 3명을 상대로 모두 6차례에 걸쳐 227만원의 소액결제 피해를 입혔다. 같은 달 20일엔 고양시 일산동구에서 6차례에 걸쳐 178만 2000원의 소액 결제가 이뤄졌다. KT 피해 현황이 갈수록 달라지면서 KT 이용자 소액결제에 대한 전수 조사가 필요하다는 지적이 나온다. KT 측은 '피해 지역은 '추정 위치'로 수사를 통해 확인이 필요한 상황'이라고 해명했다. 또한 ARS 인증에 대해서만 조사한다는 지적에 대해서는 '불법 기지국 ID를 찾기 위해 ARS에 국한해 조사했고 이 불법 ID 신호를 수신한 2만 명을 대상으로는 모든 인증 수단에 대해 조사했다'고 설명했다."

result = chain.run(input=test_news)

print(result)



> Entering new LLMChain chain...
Prompt after formatting:
System: 당신은 뉴스 키워드 추출 전문가입니다. 뉴스 기사에서 핵심 키워드 3개를 뽑으세요. 항상 '키워드: A, B, C' 형식을 따르세요.
Human: 삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다.
AI: 키워드: 삼성전자, 인공지능, 엔비디아
Human: 세계보건기구(WHO)는 최근 새로운 건강 위기에 대응하기 위해 국제 협력의 중요성을 강조했다.
AI: 키워드: 세계보건기구, 건강위기, 국제협력
Human: 테슬라는 차세대 전기차 배터리를 공개하며, 주행 거리와 충전 속도를 획기적으로 개선했다고 밝혔다.
AI: 키워드: 테슬라, 전기차, 배터리
Human: KT 무단 소액결제 피해가 당초 알려진 대로 서울 서남권·경기 일부 지역을 넘어 서울 서초구·동작구, 경기 고양시 일산동구 등에서도 일어난 것으로 뒤늦게 파악됐다. 20일 KT가 국회 과학기술정보방송통신위원회 소속 황정아 의원(더불어민주당)에게 제출한 인증 시간 기준 피해 지역 자료에 따르면 알려진 곳들 외에도 서울 동작구, 서초구, 고양시 일산동구가 포함돼 있다. KT가 처음 피해가 발생한 시점으로 지목한 지난달 5일 서울 동작구 일대에서 2건에 걸쳐 87만 9500원의 무단 결제가 발생했다. 이어 8일과 11일에는 서울 서초구에서 3명을 상대로 모두 6차례에 걸쳐 227만원의 소액결제 피해를 입혔다. 같은 달 20일엔 고양시 일산동구에서 6차례에 걸쳐 178만 2000원의 소액 결제가 이뤄졌다. KT 피해 현황이 갈수록 달라지면서 KT 이용자 소액결제에 대한 전수 조사가 필요하다는 지적이 나온다. KT 측은 '피해 지역은 '추정 위치'로 수사를 통해 확인이 필요한 상황'이라고 해명했다. 또한 ARS 인증에 대해서만 조사한다는 지적에 대해서는 '불법 기지국 ID를 찾기 위해 ARS에 국한해 조사했고 이 불법 ID 신호를 수신한 2만 명을 대상으